# MADE. Advanced ML. Home assignment #3

## The Adventure of the Dancing Men

<img src="./data/img/dancing_men.png" width="600" align="left">

### 1.
Реализуйте базовый частотный метод по Шерлоку Холмсy

- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом.

In [188]:
from collections import Counter
from itertools import product
import os
import random
import re
from typing import Dict, Optional, List, Tuple

import matplotlib.pyplot as plt; plt.ion()
import numpy as np
from tqdm.notebook import tqdm

In [2]:
CORPUS_DATAPATH = "data/corpora"
WAR_AND_PEACE_RU_FILEPATH = os.path.join(CORPUS_DATAPATH, "WarAndPeace.txt")
WAR_AND_PEACE_EN_FILEPATH = os.path.join(CORPUS_DATAPATH, "WarAndPeaceEng.txt")
ANNA_KARENINA_RU_FILEPATH = os.path.join(CORPUS_DATAPATH, "AnnaKarenina.txt")

In [3]:
def read_text(filepath: str) -> str:
    with open(filepath, 'r') as f:
        text = f.read()
        text = text.replace('\n', ' ')
    return text

In [4]:
def preprocess_text(text: str, languare: str = 'ru') -> str:
    """Process text with following steps:
    1. To lower case
    2. Take only letters from appropriate languare
    3. Replace sequence of subspaces to one
    """
    processed = text.lower()
    if languare == "ru":
        pattern_to_delete = re.compile("[^а-яё ]")
    elif languare == "en":
        pattern_to_delete = re.compile("[^a-z ]")
    else:
        raise ValueError(f"Unknown languale: {languare}")
    processed = re.sub(pattern_to_delete, " ", processed)
    processed = re.sub(r"\s+", " ", processed)
    return processed

In [5]:
CORPUS = dict()

for text_id, text_filepath, lang in tqdm(zip(
    ["war_and_peace_ru", "war_and_peace_en", "anna_karenina_ru"],
    [WAR_AND_PEACE_RU_FILEPATH, WAR_AND_PEACE_EN_FILEPATH, ANNA_KARENINA_RU_FILEPATH],
    ["ru", "en", "ru"]
)):
    text_i = read_text(text_filepath)
    CORPUS[text_id] = preprocess_text(text_i, lang)

In [6]:
for text_id, text_processed in CORPUS.items():
    print(text_id)
    print(text_processed[:100], "\n")

war_and_peace_ru
 война и мир самый известный роман льва николаевича толстого как никакое другое произведение писател 

war_and_peace_en
 the project gutenberg ebook of war and peace by leo tolstoy this ebook is for the use of anyone any 

anna_karenina_ru
 анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой вс 



- Частоты по корпусам

In [7]:
def calculate_letter_frequencies(text: str) -> Dict[str, float]:
    """Calculate relative frequencies of characters from text"""
    letter_counter = Counter(text)
    letter_counter = {i: j / len(text) for i, j in letter_counter.items()}
    letter_counter = dict(
        sorted(
            letter_counter.items(),
            key=lambda x: x[1],
            reverse=True
        )
    )
    return letter_counter

- Зашифруем и попробуем расшифровать тестовые данные случайной перестановкой символов

In [8]:
def get_alphabet(language: str) -> List[str]:
    """Generate list of letters for appripriate languare"""
    if language == "ru":
        alphabet = [chr(i) for i in range(ord('а'), ord('я') + 1)] + ['ё', " "]
    elif language == "en":
        alphabet = [chr(i) for i in range(ord('a'), ord('z') + 1)] + [" "]
    else:
        raise ValueError(f"Unknown languale: {languare}")
    return alphabet


def get_shuffled_encoder(language: str = "ru") -> Dict[str, str]:
    """Provide mapping from original alphabet to shuffled one"""
    mapping = dict()
    alphabet = get_alphabet(language)
    new_alphabet = alphabet.copy()
    random.shuffle(new_alphabet)
    return dict(zip(alphabet, new_alphabet))


def encode_text(text: str, encoder: Dict[str, str]) -> str:
    """Encode text based on encoder latters mapping"""
    result = ""
    for i in text:
        result += encoder.get(i, "?")
    return result


def get_optimal_decoder(encoded_text: str, train_freq: Dict[str, str]) -> Dict[str, str]:
    """Optimal decoder from frequencies"""
    test_freq = calculate_letter_frequencies(encoded_text)
    decoder = dict(zip(test_freq, train_freq))
    return decoder


def decode_text(
    encoded_text: str,
    decoder: Dict[str, str],
    missing_value: str = "?"
) -> str:
    """Decode text using optimal decoder previously calculated"""
    result = ""
    for i in encoded_text:
        result += decoder.get(i, missing_value)
    return result


def calculate_accuracy(text_true, text_pred):
    """Accuracy score of exact letters matching"""
    assert len(text_true) == len(text_pred), "Text lengths should match exactly"
    return sum([i == j for i, j in zip(text_true, text_pred)]) / len(text_true)

In [9]:
random_encoder = get_shuffled_encoder("ru")

In [10]:
train_freq = calculate_letter_frequencies(CORPUS['war_and_peace_ru'])

In [11]:
# Пример из Train dataset-а
test_text = CORPUS['war_and_peace_ru']
test_text_encoded = encode_text(test_text, random_encoder)
optimal_decoder = get_optimal_decoder(test_text_encoded, train_freq)
test_text_decoded = decode_text(
    test_text_encoded,
    optimal_decoder
)

print_size = 51
print("Actual:".ljust(10), test_text[:print_size])
print("Encoded:".ljust(10), test_text_encoded[:print_size])
print("Decoded:".ljust(10), test_text_decoded[:print_size])
print(f"Accuracy on whole text: {calculate_accuracy(test_text, test_text_decoded)}")

Actual:     война и мир самый известный роман льва николаевича
Encoded:   зфч сюзхзлхозвюлщ зхьфмвгсщ зочлюсзуйфюзсхцчуюмфхню
Decoded:    война и мир самый известный роман льва николаевича
Accuracy on whole text: 1.0


In [12]:
# Пример из Test dataset-а, где частоты посчитаны на целом тексте:
test_text = CORPUS['anna_karenina_ru']
test_text_encoded = encode_text(test_text, random_encoder)
optimal_decoder = get_optimal_decoder(test_text_encoded, train_freq)
test_text_decoded = decode_text(
    test_text_encoded,
    optimal_decoder
)

print_size = 61
print("Actual:".ljust(10), test_text[:print_size])
print("Encoded:".ljust(10), test_text_encoded[:print_size])
print("Decoded:".ljust(10), test_text_decoded[:print_size])
print(f"Accuracy on whole text: {calculate_accuracy(test_text, test_text_decoded):.4f}")

Actual:     анна каренина один из самых знаменитых романов льва толстого
Encoded:   зюссюзцюомсхсюзчрхсзхьзвюлщязьсюлмсхгщязочлюсчфзуйфюзгчувгчбч
Decoded:    еиие кераиние одни нч семьх чиемаинтьх ромеиов лгве толстоыо
Accuracy on whole text: 0.6291


### 2.

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:

- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм

In [13]:
def calculate_bigram_frequencies(text: str) -> Dict[str, float]:
    """Calculate relative frequencies of bigrams from text"""
    bigram_counter = Counter()
    for i in range(len(text) - 1):
        bigram_counter[text[i: i + 2]] += i
    
    n_bigrams = sum(bigram_counter.values())
    bigram_counter = dict(
        sorted(
            bigram_counter.items(),
            key=lambda x: x[1],
            reverse=True
        )
    )
    bigram_counter = {i: j / n_bigrams for i, j in bigram_counter.items()}
    return bigram_counter

In [14]:
# abc -> ab bc
# ab bc -> abc

In [57]:
# Calculate bigram frequencies using War and Peace [RU] as train dataset:
train_bigram_freq = calculate_bigram_frequencies(CORPUS['war_and_peace_ru'])

In [58]:
test_text_encoded = encode_text(CORPUS['anna_karenina_ru'], random_encoder)
test_bigram_freq = calculate_bigram_frequencies(test_text_encoded)

In [59]:
list(train_bigram_freq.items())[:10]

[('о ', 0.021671493454038303),
 ('и ', 0.018108026579692883),
 ('е ', 0.015485657194981196),
 ('а ', 0.015362732805954401),
 (' с', 0.015187212825279217),
 (' п', 0.015011360591923206),
 (' в', 0.014898685499327182),
 (' н', 0.014874541819007736),
 ('то', 0.013590255845312203),
 (' о', 0.012039298756919049)]

In [60]:
list(test_bigram_freq.items())[:10]

[('чз', 0.02382762636749498),
 ('мз', 0.018935873797493535),
 ('юз', 0.01851984451320598),
 ('хз', 0.017957561175334535),
 ('зс', 0.016333824866984408),
 ('зв', 0.01606188531157267),
 ('зф', 0.014739839768844788),
 ('гч', 0.014256806184461339),
 ('зы', 0.013971276602894347),
 ('зч', 0.01349492169970542)]

In [74]:
# Get optimal bigram decoder based on bigram frequesncies:
bigram_decoder = dict()
for test_bg, train_bg in zip(test_bigram_freq.keys(), train_bigram_freq.keys()):
    if test_bg[0] not in bigram_decoder and train_bg[0] not in bigram_decoder.values():
        bigram_decoder.update({test_bg[0]: train_bg[0]})
    if test_bg[1] not in bigram_decoder and train_bg[1] not in bigram_decoder.values():
        bigram_decoder.update({test_bg[1]: train_bg[1]})

# add missing letters:
missing_letters = dict(zip(
    [i for i in get_alphabet("ru") if i not in bigram_decoder],
    [i for i in get_alphabet("ru") if i not in bigram_decoder.values()]
))
bigram_decoder.update(missing_letters)

In [75]:
# Пример из Test dataset-а, где частоты посчитаны на целом тексте:
test_text_decoded = decode_text(
    test_text_encoded,
    bigram_decoder
)

print_size = 61
print("Actual:".ljust(10), test_text[:print_size])
print("Encoded:".ljust(10), test_text_encoded[:print_size])
print("Decoded:".ljust(10), test_text_decoded[:print_size])
print(f"Accuracy on whole text: {calculate_accuracy(test_text, test_text_decoded):.4f}")

Actual:     анна каренина один из самых знаменитых романов льва толстого
Encoded:   зюссюзцюомсхсюзчрхсзхьзвюлщязьсюлмсхгщязочлюсчфзуйфюзгчувгчбч
Decoded:    ессе йенисасе отас аз пемых зсемисауых номесов льве уолпуого
Accuracy on whole text: 0.4765


- Качество ухудшилось... Перепробовал несколько вариантов, как именно считать decoder на основани биграмм. В данном подходе, где просто поочередно перебираются подряд самые популярные биграммы, удалось достигнуть качества лишь __47.65%__. Другие попытки оказались еще менее удачными

### 3.

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.


Следующий символ в тексте зависит от предыдущего символа. Значит, рассмотрим текст как цепь Маркова.<br>
Применим MCMC-семплирование. На каждой итерации меняем местами пару символов в текущей перестановке, считаем правдоподобие восстановления текста из текущей пререстановки, принимаем новую перестновку с определённой вероятностью. Выберем лучшую попытку.

In [288]:
def generate_permutation(alphabet: List[str]) -> np.ndarray:
    """Generate random permutation from language specific alphabet"""
    return np.random.permutation(alphabet)

In [289]:
def make_inplace_swap(permutation: np.ndarray) -> Tuple[int, int]:
    """Swap 2 random charecters from permutation"""
    a, b = np.random.choice(len(permutation), size=2, replace=False)
    permutation[a], permutation[b] = permutation[b], permutation[a]
    return a, b

In [320]:
def apply_permutation(text: str, permutation: np.ndarray, alphabet: List[str] = ru_alphabet) -> str:
    """Encode text with permutation"""
    return text.translate(
        str.maketrans(''.join(alphabet), ''.join(permutation))
    )

def get_transition_matrix(text: str) -> np.ndarray:
    transition_matrix = np.zeros(
        (
            len(char_to_idx), len(char_to_idx)
        )
    )
    for i in range(len(text) - 1):
        transition_matrix[char_to_idx[text[i]], char_to_idx[text[i + 1]]] += 1
    transition_matrix = np.clip(transition_matrix, a_min=1, a_max=None)
    transition_matrix = (
        np.log(transition_matrix).T - np.log(transition_matrix.sum(axis=1))
    ).T
    return transition_matrix

def calculate_log_likelihood(text: str, permutation: np.ndarray, alphabet: List['str']) -> float:
    # encode text with new permutation:
    text = apply_permutation(text, permutation, alphabet)
    # provide calculation:
    result = 0
    for i in range(len(text) - 1):
        result += transition_matrix[
            char_to_idx[text[i]], char_to_idx[text[i + 1]]
        ]
    return result

def get_optimal_permutation_mcmc(text: str, alphabet: List[str], n_iter: int) -> np.ndarray:
    permutation = generate_permutation(alphabet)
    best_permutation = permutation.copy()
    log_likelihood = calculate_log_likelihood(text, permutation, alphabet)
    log_likelihood_best = log_likelihood
    
    for i in tqdm(range(n_iter)):
        id_a, id_b = make_inplace_swap(permutation)
        log_likelihood_i = calculate_log_likelihood(text, permutation, alphabet)
        if log_likelihood_i >= log_likelihood:
            log_likelihood = log_likelihood_i
            if log_likelihood_i > log_likelihood_best:
                log_likelihood_best = log_likelihood_i
                best_permutation = permutation.copy()
        else:
            if random.random() < np.exp(log_likelihood_i - log_likelihood):
                log_likelihood = log_likelihood_i
            else:
                permutation[id_a], permutation[id_b] = permutation[id_b], permutation[id_a]
    return best_permutation

In [325]:
test_example = test_text[:995]
test_example_encoded = encode_text(test_example, random_encoder)
test_example_encoded

'зюссюзцюомсхсюзчрхсзхьзвюлщязьсюлмсхгщязочлюсчфзуйфюзгчувгчбчзсюнхсюмгвёзвгюфем зютчохьлчлзтоюьч зфвмзвнювгухфщмзвмлйхзычячдхзрожбзсюзрожбюзцюдрюёзсмвнювгухфюёзвмлйёзсмвнювгухфюзычзвфчмлжзэгчзцсхбюзчзфмнсщязпмссчвгёязчзуаъфхзчзфмомзчзвмлймзчзнмучфмнмвцчлзрчвгчхсвгфмзумфзгчувгч очлюсзехочцчбчзрщяюсхёзнювгйзымофюёзумфзгчувгч зюссюзцюомсхсюзочлюсзехочцчбчзрщяюсхёзюссюзцюомсхсюзычоюьхуюзвчфомлмссхцчфзфвмрсмфсчвгйазвчрмодюсхёзсмчъщню сюёзвфчъчрюзоювцчфюссчвгйзычфмвгфчфюсхёзжрхфхгмуйсчзвчнмгюухвйзфзэгчлзочлюсмзвзпмуйсчвгйазяжрчдмвгфмссчбчзфьбуёрюзюфгчоюзсюздхьсйзчсзфщвгжыюузьрмвйзцюцзяжрчдсхцзхзлщвухгмуйзхзсюьсюнмсхмзхвцжввгфюзфхрмузсмзфзгчлзнгчъщзсмчвычохлчзоюьомехгйзфчыочвзюзфзгчлзнгчъщзьювгюфхгйзуаъхгйздхьсйзфзъмвнхвумссщязсхцчбрюзсмзхвгчшхлщязфвмязммзыочёфумсхёязфзмзбчрщзчрхсзлювгхгщ зыхвюгмуйзычзфхрхлчлжзбчснюочфзвцюьюузрчвгчмфвцчлжзэгчзфмшйзсмвущяюссюёзэгчзфмшйзымофюёзцгчзжзсювзхьзыхвюгмум злчдмгзычоюфсёгйвёзвзэгхлзюзфзмфочымзцгчзыомрвгюфхгзячгйзнгчзсхъжрйзычрчъсчмзтзлзрчвгчмфвцх '

In [326]:
char_to_idx = {char: idx for idx, char in enumerate(ru_alphabet)}
transition_matrix = get_transition_matrix(CORPUS['war_and_peace_ru'])
best_permutation_mcmc = get_optimal_permutation_mcmc(test_example_encoded, ru_alphabet, n_iter=10000)

In [327]:
apply_permutation(test_example_encoded, best_permutation_mcmc, ru_alphabet)

' анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива по своему это книга о вечных ценностях о любви о вере о семье о человеческом достоинстве лев толстойроман широкого дыхания часть первая лев толстой анна каренина роман широкого дыхания анна каренина поразила современников вседневностью содержания необычайная свобода раскованность повествования удивительно сочетались в этом романе с цельностью художественного взгляда автора на жизнь он выступал здесь как художник и мыслитель и назначение искусства видел не в том чтобы неоспоримо разрешить вопрос а в том чтобы заставить любить жизнь в бесчисленных никогда не истощимых всех ее проявлениях в е годы один маститый писатель по видимому гончаров сказал достоевскому это вещь неслыханная это вещь первая кто у нас из писателей может поравняться с этим а в европе кто представит хоть что нибудь подобное ф м достоевский'

In [328]:
calculate_accuracy(
    test_example,
    apply_permutation(test_example_encoded, best_permutation_mcmc, ru_alphabet)
)

1.0

### 4.

Расшифруйте сообщение:

←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸ ↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊ ↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒ ↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝← ⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏

Или это (они одинаковые, второй вариант просто на случай проблем с юникодом):

დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭ􏰀ႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭ􏰀ႣჵისႼჰႨჲდႩჳჲႨ􏰀ႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨ ႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵ􏰀ႧჂჲდႨ􏰀ႣႩჳჂ􏰀ႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხს დდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩ􏰀ႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ

In [384]:
secret_message = ("←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸ "
"↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊ ↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛"
"⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒ ↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝← ⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏")

secret_message_alphabet = sorted(set(secret_message))
print(f"Contains: {len(secret_message_alphabet)} unique charecters")

# # add some letters
# secret_message_alphabet += ['a', 'b', 'c', 'd', 'f']
# assert len(secret_message_alphabet) == len(ru_alphabet)

Contains: 29 unique charecters


In [385]:
char_to_idx = {char: idx for idx, char in enumerate(ru_alphabet)}
transition_matrix = get_transition_matrix(CORPUS['war_and_peace_ru'])

alphabet = list(train_freq.keys())[:len(secret_message_alphabet)]
secret_message_encoded = encode_text(
    secret_message,
    encoder=dict(zip(secret_message_alphabet, alphabet))
)

print(secret_message_encoded)

ойытдкядктстлодибжрхыпиямдтытдюбчлтдибжрхыпиямдлогйлдздц лбебдйббвуоитндгблбжямдыоегбдюжбчтлхлпдйгбжоодкйоебдкя дкйодйсоыхытдюжхктыпибдтдюбызчтлодрхгйтрхыпиямдвхыы дшхдюбйыосиоодчолкожлбодшхсхитодгзжйхдаблндгбио чибдндитчоебдиодбвоухь


In [386]:
best_permutation_mcmc = get_optimal_permutation_mcmc(secret_message_encoded, alphabet, n_iter=100000)

In [387]:
apply_permutation(secret_message_encoded, best_permutation_mcmc, alphabet)

'если вы вимите норжальный или почти норжальный текст у зютого соодцения который легко прочитать скорее всего выю все смелали правильно и получите жаксижальный даллю ба послемнее четвертое бамание курса хотя конеючно я ничего не одецаш'

<hr>